In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import fitz

from pprint import pprint

import util
import lines
import group
import label
import records

In [2]:
test_files_dir = ".././test_files/"
test_files = util.list_files(test_files_dir)
test_files

['.././test_files/LS_index_1937.pdf',
 '.././test_files/LS_index_1986_split.pdf',
 '.././test_files/Argentinia_LS_index_1934_test_I_DE.pdf',
 '.././test_files/LS_index_1976.pdf',
 '.././test_files/LS_index_1944.pdf',
 '.././test_files/LS_index_1920_split.pdf',
 '.././test_files/LS_index_1986.pdf',
 '.././test_files/LS_index_1920.pdf',
 '.././test_files/LS_index_1944_split.pdf']

In [3]:
p_start = 1
pages, dicts = util.read_pdf(test_files[-2], p_start)

l_df = lines.make_lines_df(dicts)
l_df = lines.merge_close_lines(l_df)
l_df = lines.remove_useless_lines(l_df)

display(l_df)

Reading pdf from .././test_files/LS_index_1920.pdf
...
Finished reading 11 page(s)


,line_text,spans,x0,y0,x1,y1,page
0,I. CHRONOLOGICAL INDEX OF LAWS AND ORDERS IN,[I. CHRONOLOGICAL INDEX OF LAWS AND ORDERS IN],48.0,78.66,351.96,90.66,1
1,THE ENGLISH EDITION OF THE LEGISLATiVE,[THE ENGLISH EDITION OF THE LEGISLATiVE],75.0,91.03,351.87,104.13,1
2,SERIES OF THE INTERNATIONAL LABOUR,"[SERIES, OF, THE, INTERNATIONAL, LABOUR]",77.0,105.66,350.81,116.86,1
3,"OFFICE, 1920.","[OFFICE, 1920.]",76.0,118.41,146.62,129.81,1
4,"Nora.—T. E., S., or Ti. after the title of a l...","[Nora.—T. E., S., or Ti. after the title of a ...",63.0,147.96,350.17,157.96,1
...,...,...,...,...,...,...,...
815,Decree to include messengers under the Eight-H...,"[Decree, to include messengers under the Eight...",68.0,511.51,327.87,522.03,11
816,"October, 1920. T., Ur. 3—5 ..","[October, 1920., T.,, Ur. 3—5, ..]",85.0,520.81,225.06,531.55,11
819,. Act respecting occupational accidents. 15th ...,"[., Act, respecting occupational accidents., 1...",68.0,526.76,330.57,537.86,11
820,Ur.r .. .. .. .. .. .. .. ..,"[Ur.r, .., .., .., .., .., .., .., ..]",85.0,535.33,301.28,546.43,11


In [4]:
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(l_df)
borders = lines.make_borders_df(bins_x0, bins_x1)
display(x0_n, bins_x0, bins_x1)

3

,x0,lines,last_x0,count,page
4,"[48.0, 46.0, 46.0, 46.0, 47.0, 47.0]","[0, 5, 6, 8, 9, 10]",47.0,6,1
3,"[75.0, 77.0, 76.0, 77.0, 77.0, 77.0, 77.0, 77....","[1, 2, 3, 12, 14, 16, 20, 23, 27, 30, 35, 41, ...",79.0,15,1
1,"[93.0, 95.0, 94.0, 94.0, 94.0, 94.0, 95.0, 95....","[13, 15, 17, 18, 19, 21, 22, 24, 25, 26, 28, 2...",97.0,29,1
6,48.0,58,48.0,1,2
5,"[65.0, 65.0, 64.0, 64.0, 64.0, 64.0, 64.0, 64....","[59, 63, 67, 70, 74, 77, 79, 84, 89, 93, 98, 1...",65.0,20,2
4,"[82.0, 81.0, 82.0, 82.0, 82.0, 81.0, 81.0, 81....","[60, 61, 62, 64, 68, 69, 71, 72, 73, 75, 76, 7...",82.0,47,2
2,"[63.0, 63.0, 63.0, 64.0, 64.0, 63.0]","[146, 151, 203, 205, 211, 213]",63.0,6,3
1,"[80.0, 81.0, 80.0, 81.0, 80.0, 81.0, 80.0, 80....","[147, 152, 155, 158, 164, 168, 171, 173, 178, ...",80.0,21,3
0,"[97.0, 97.0, 97.0, 98.0, 97.0, 98.0, 97.0, 97....","[148, 149, 150, 153, 156, 157, 159, 160, 161, ...",97.0,38,3
4,"[47.0, 47.0, 51.0, 52.0, 51.0]","[217, 220, 242, 253, 265]",51.0,5,4


,x1,lines,last_x1,count,page
12,"[351.96, 351.87, 350.81, 350.17, 350.91, 353.1...","[0, 1, 2, 4, 5, 7, 8, 9]",351.21,8,1
13,"[326.6, 325.58, 325.0, 323.66, 324.37, 324.22,...","[59, 60, 61, 63, 67, 68, 69, 70, 71, 72, 73, 7...",326.34,52,2
13,"[341.06, 339.73, 340.39, 340.0, 339.43, 338.25...","[147, 148, 149, 152, 155, 156, 158, 159, 160, ...",342.83,37,3
13,"[324.84, 325.46, 324.03, 324.24, 325.47, 325.2...","[218, 223, 225, 227, 228, 230, 232, 235, 236, ...",328.5,39,4
11,"[332.46, 332.55, 333.8, 335.3, 335.15, 333.95,...","[293, 294, 296, 299, 301, 303, 305, 307, 309, ...",336.08,44,5
15,"[327.07, 327.18, 326.61, 327.56, 326.43, 327.5...","[372, 373, 374, 376, 377, 379, 380, 381, 382, ...",326.8,40,6
15,"[340.36, 339.99, 340.58, 340.65, 339.75, 340.1...","[454, 456, 459, 461, 462, 463, 467, 469, 470, ...",335.19,42,7
15,"[335.36, 336.1, 335.11, 335.82, 335.4, 335.74,...","[532, 534, 535, 537, 539, 543, 544, 545, 546, ...",334.12,37,8
14,"[327.22, 324.88, 327.53, 325.56, 327.53, 325.8...","[603, 604, 606, 607, 609, 610, 614, 615, 616, ...",327.63,38,9
11,"[333.14, 331.52, 331.9, 332.16, 330.34, 330.28...","[684, 688, 689, 690, 692, 693, 695, 696, 699, ...",329.71,38,10


In [5]:
borders

,page,x0,x1,dx
0,1,46.666667,351.320000,304.653333
1,2,48.000000,325.162115,277.162115
2,3,63.333333,340.336757,277.003423
3,4,49.600000,327.440000,277.840000
4,5,57.000000,334.928182,277.928182
5,6,49.333333,327.321250,277.987917
6,7,62.000000,337.993571,275.993571
7,8,57.500000,334.678108,277.178108
8,9,49.000000,326.284474,277.284474
9,10,53.833333,330.460789,276.627456


In [6]:
li_df = label.assign_types(l_df, bins_x0, bins_x1, x0_n)
l = label.assign_labels(li_df, x0_n)
li_df.head(20)

,line_text,spans,x0,y0,x1,y1,page,x0_type,x1_type
0,I. CHRONOLOGICAL INDEX OF LAWS AND ORDERS IN,[I. CHRONOLOGICAL INDEX OF LAWS AND ORDERS IN],48.0,78.66,351.96,90.66,1,0,2
1,THE ENGLISH EDITION OF THE LEGISLATiVE,[THE ENGLISH EDITION OF THE LEGISLATiVE],75.0,91.03,351.87,104.13,1,1,2
2,SERIES OF THE INTERNATIONAL LABOUR,"[SERIES, OF, THE, INTERNATIONAL, LABOUR]",77.0,105.66,350.81,116.86,1,1,2
3,"OFFICE, 1920.","[OFFICE, 1920.]",76.0,118.41,146.62,129.81,1,1,0
4,"Nora.—T. E., S., or Ti. after the title of a l...","[Nora.—T. E., S., or Ti. after the title of a ...",63.0,147.96,350.17,157.96,1,-1,2
5,"the full text, an extract, a summary, or the t...","[the full text, an extract, a summary, or the ...",46.0,155.88,350.91,165.98,1,0,2
6,given in the Series.,[given in the Series.],46.0,164.33,118.83,173.83,1,0,0
7,The letters and unbracketej numbers given afte...,"[The letters and unbracketej numbers, given af...",64.0,171.43,353.15,182.24,1,-1,2
8,"references in the Legislative Series for 5920,...",[references in the Legislative Series for 5920...,46.0,180.63,350.48,190.03,1,0,2
9,of the documents in question or in referring t...,[of the documents in question or in referring ...,47.0,188.46,351.21,198.06,1,0,2


In [7]:
cli_df = label.correct_x0_types(li_df, bins_x0, bins_x1, x0_n)
ll_df = label.assign_labels(cli_df, x0_n)

lli_df = ll_df.copy()
lli_df["new_label"] = lli_df["label"]
lli_df = label.improve_country_classification(lli_df)
#ll_df.to_csv("labeling_1986_full.csv")

lli_df

,line_text,spans,x0,y0,x1,y1,page,x0_type,x1_type,label,new_label
0,I. CHRONOLOGICAL INDEX OF LAWS AND ORDERS IN,[I. CHRONOLOGICAL INDEX OF LAWS AND ORDERS IN],48.0,78.66,351.96,90.66,1,0,2,other,other
1,THE ENGLISH EDITION OF THE LEGISLATiVE,[THE ENGLISH EDITION OF THE LEGISLATiVE],75.0,91.03,351.87,104.13,1,1,2,start,start
2,SERIES OF THE INTERNATIONAL LABOUR,"[SERIES, OF, THE, INTERNATIONAL, LABOUR]",77.0,105.66,350.81,116.86,1,1,2,start,start
3,"OFFICE, 1920.","[OFFICE, 1920.]",76.0,118.41,146.62,129.81,1,1,0,other,other
4,"Nora.—T. E., S., or Ti. after the title of a l...","[Nora.—T. E., S., or Ti. after the title of a ...",63.0,147.96,350.17,157.96,1,-1,2,other,other
...,...,...,...,...,...,...,...,...,...,...,...
815,Decree to include messengers under the Eight-H...,"[Decree, to include messengers under the Eight...",68.0,511.51,327.87,522.03,11,1,2,start,start
816,"October, 1920. T., Ur. 3—5 ..","[October, 1920., T.,, Ur. 3—5, ..]",85.0,520.81,225.06,531.55,11,2,0,end,end
819,. Act respecting occupational accidents. 15th ...,"[., Act, respecting occupational accidents., 1...",68.0,526.76,330.57,537.86,11,1,2,start,start
820,Ur.r .. .. .. .. .. .. .. ..,"[Ur.r, .., .., .., .., .., .., .., ..]",85.0,535.33,301.28,546.43,11,2,1,end,end


In [8]:
lli_df.loc[ll_df["page"]==1].head(30)

,line_text,spans,x0,y0,x1,y1,page,x0_type,x1_type,label,new_label
0,I. CHRONOLOGICAL INDEX OF LAWS AND ORDERS IN,[I. CHRONOLOGICAL INDEX OF LAWS AND ORDERS IN],48.0,78.66,351.96,90.66,1,0,2,other,other
1,THE ENGLISH EDITION OF THE LEGISLATiVE,[THE ENGLISH EDITION OF THE LEGISLATiVE],75.0,91.03,351.87,104.13,1,1,2,start,start
2,SERIES OF THE INTERNATIONAL LABOUR,"[SERIES, OF, THE, INTERNATIONAL, LABOUR]",77.0,105.66,350.81,116.86,1,1,2,start,start
3,"OFFICE, 1920.","[OFFICE, 1920.]",76.0,118.41,146.62,129.81,1,1,0,other,other
4,"Nora.—T. E., S., or Ti. after the title of a l...","[Nora.—T. E., S., or Ti. after the title of a ...",63.0,147.96,350.17,157.96,1,-1,2,other,other
5,"the full text, an extract, a summary, or the t...","[the full text, an extract, a summary, or the ...",46.0,155.88,350.91,165.98,1,0,2,other,other
6,given in the Series,[given in the Series.],46.0,164.33,118.83,173.83,1,0,0,country,region
7,The letters and unbracketej numbers given afte...,"[The letters and unbracketej numbers, given af...",64.0,171.43,353.15,182.24,1,-1,2,other,other
8,"references in the Legislative Series for 5920,...",[references in the Legislative Series for 5920...,46.0,180.63,350.48,190.03,1,0,2,other,other
9,of the documents in question or in referring t...,[of the documents in question or in referring ...,47.0,188.46,351.21,198.06,1,0,2,other,other


In [9]:
rec_gr = records.group_records(lli_df)
rec = records.extract_records(lli_df)
rec

,country,region,text,page
0,AUSTRJA,,Administrative Instruction of the State Depart...,2
1,AUSTRJA,,Act respecting domestic servants' contracts (D...,2
2,AUSTRJA,,Administrative Instruction of the State Depart...,2
3,AUSTRJA,,Administrative Instruction of the State Depart...,2
4,AUSTRJA,,Administrative Instruction of the State Depart...,2
...,...,...,...,...
207,URUGIJAY,,Resolution concerning ocdupational accidents i...,11
208,URUGIJAY,,Decree issuing new regulations under the Act r...,11
209,URUGIJAY,,Decree respecting the supervision of hours of ...,11
210,URUGIJAY,,Decree to include messengers under the Eight-H...,11


In [26]:
rec.to_csv("index_1920.csv")

In [55]:
classification.improve_classification(ll_df)

,line_text,spans,x0,y0,x1,y1,page,x0_type,x1_type,label,new_label,date
0,1.—LIST OF LAWS AND ORDERS OF 1944 NOT PUBLISH...,[1.—LIST OF LAWS AND ORDERS OF 1944 NOT PUBLIS...,35.0,94.69,342.19,104.79,3,1,2,start,other,
1,"LEGISLATIVE SERIES FOR THAT YEAR WHICH AMEND, RE","[LEGISLATIVE SERIES FOR THAT YEAR WHICH AMEND,...",48.0,104.29,336.14,114.79,3,-1,1,other,other,
2,PEAL OR SUPERSEDE LA\VS AND ORDERS PUBLISHED PRE,[PEAL OR SUPERSEDE LA\VS AND ORDERS PUBLISHED ...,49.0,114.39,338.64,124.89,3,-1,2,other,other,
3,VIOUSLY IN THE LEGISLATIVE SERIES,[VIOUSLY IN THE LEGISLATIVE SERIES],48.0,124.46,236.91,134.86,3,-1,1,other,other,
4,ARGENTINE REPUBLIC,[ARGENTINE REPUBLIC],26.0,147.26,151.34,158.86,3,0,0,country,country,
...,...,...,...,...,...,...,...,...,...,...,...,...
4320,Apr. 28 L.D. respecting the organisation of la...,"[Apr. 28, L.D., respecting the organisation of...",62.0,528.92,369.30,539.32,82,1,2,start,start,Apr. 28
4321,agreements; labour courts]. No. 311. (Zb. pp. ...,"[agreements; labour courts]., No. 311., (Zb. p...",109.0,537.12,337.55,547.49,82,2,1,end,end,
4322,G.O. = Gacefa Oficial de los Unmdos de Venezuela.,"[G.O. =, Gacefa, Oficial de los, Unmdos de Ven...",63.0,552.82,296.01,563.62,82,1,1,other,other,
4323,Zb. = Zbornile zakona i naredaba.,"[Zb. =, Zbornile, zakona, i, naredaba.]",62.0,562.45,195.35,573.75,82,1,0,other,other,


## Probleme
- Dokumente mit 2 Spalten
- Dokumente mit Länder-Überschrift über ganze Breite der Seite
- Erkennen, ab welcher Seite die eigentlichen Indexes beginnen
- Unnötigen Anfangstext auf der ersten Seite rausfiltern
    - 1920

## Fragen
